<a href="https://colab.research.google.com/github/FKz11/Neural_Network/blob/lesson-4/lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## lesson-4

Библиотеки

In [10]:
import numpy as np
from tensorflow.random import set_seed

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt

In [11]:
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

In [12]:
# загрузка тренировочных и тестовых данных
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# конвертация чисел из uint8 в float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10)

In [13]:
def nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, start_kernel = 2, 
                              neurons = 2000, batch_size = 128, epochs = 7, verbose = 1, seed = 42):

    set_seed(seed)
    
    first_layer = np.empty(branch, dtype=object)
    
    first_input = layers.Input(shape=x_train.shape[1:]) # shape = (32, 32, 3)

    for i in range(branch):
      if i > 1:                                                                                         
        conv = layers.Conv2D(start_filters*2**(i-1),2*(branch-1) + start_kernel - 2*i,                            # filters = 12, 12, 24, 48
                                            activation='relu',padding = 'same',name=f'conv_1_{i+1}')(first_input) # kernel_size = 8, 6, 4, 2
        first_layer[i] = layers.MaxPool2D(pool_size=(2**(i-1),2**(i-1)),name=f'pool_1_{i+1}')(conv)               # pool_size = 1, 1, 2, 4
      else:
        first_layer[i] = layers.Conv2D(start_filters,2*(branch-1) + start_kernel - 2*i,
                                       activation='relu',padding = 'same',name=f'conv_1_{i+1}')(first_input)

    layer_list = first_layer.copy()

    for i in range(branch-1):
      merge = layers.Concatenate(  axis =-1,name=f'merge_{i+2}' )([layer_list[i], first_layer[i+1]])                                                   
      conv = layers.Conv2D(start_filters*2**(i+1),2*(branch-1) + start_kernel - 2*(i+1),   # filters = 24, 48, 96
                           activation='relu',padding = 'same',name=f'conv_{i+2}')(merge)   # kernel_size = 6, 4, 2
      layer_list[i+1] = layers.MaxPool2D(pool_size=(2,2),name=f'pool_{i+2}')(conv)

    flatten = layers.Flatten(name='flatten')(layer_list[branch-1]) # input_shape = (4, 4, 96)
    
    dense_one=layers.Dense(neurons, activation='relu', name='dense_one')(flatten) # input_shape = (1536)
    
    dense_two=layers.Dense(y_train.shape[1], activation='softmax', name='dense_two')(dense_one)
    
    model = Model(inputs=first_input, outputs=dense_two)
    
    # компиляция модели
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    hh = model.fit(x_train, y_train, shuffle=True,
                   batch_size=batch_size,
                   epochs=epochs,
                   verbose=verbose,
                   validation_data=(x_test, y_test))
    
    return model, hh.history

In [ ]:
model, hh = nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, start_kernel = 2, 
                                      neurons = 2000, batch_size = 128, epochs = 9, verbose = 1, seed = 42)

Epoch 1/7
391/391 [==============================] - 42s 101ms/step - loss: 1.4674 - accuracy: 0.4691 - val_loss: 1.1182 - val_accuracy: 0.6046
Epoch 2/7
391/391 [==============================] - 38s 98ms/step - loss: 1.0247 - accuracy: 0.6367 - val_loss: 0.9408 - val_accuracy: 0.6696
Epoch 3/7
391/391 [==============================] - 38s 97ms/step - loss: 0.8276 - accuracy: 0.7096 - val_loss: 0.8681 - val_accuracy: 0.7033
Epoch 4/7
391/391 [==============================] - 38s 98ms/step - loss: 0.7018 - accuracy: 0.7539 - val_loss: 0.7921 - val_accuracy: 0.7296
Epoch 5/7
391/391 [==============================] - 38s 97ms/step - loss: 0.5868 - accuracy: 0.7942 - val_loss: 0.7956 - val_accuracy: 0.7330
Epoch 6/7
391/391 [==============================] - 38s 98ms/step - loss: 0.4748 - accuracy: 0.8334 - val_loss: 0.7975 - val_accuracy: 0.7431
Epoch 7/7
356/391 [==========================>...] - ETA: 3s - loss: 0.3628 - accuracy: 0.8721

KeyboardInterrupt: ignored

In [ ]:
{key: value[-1] for key, value in hh.items()}

In [ ]:
plot_model(model)

In [ ]:
model.summary()

Пускай количество эпох = 10. Итоговой метрикой будем считать лучшую за 10 эпох.

### branch

При увеличении количество веток увилививается как ширина нейронной сети, так и длина. Но так как после каждой свёртке идёт MaxPool с pool_size = (2,2), а начальный размер изображения 32x32, то максимальное количество веток будет 6, причём размеры конечных изображений будут 1x1 с количество фильтров равным start_filters*32.

In [ ]:
%%time

branch_list = list(range(2, 7))

metrics_branch = []

for branch in branch_list:
  metric = np.max(nn_conv(x_train, x_test, y_train, y_test, branch = branch, start_filters = 24, 
              start_kernel = 2, neurons = 2000, batch_size = 128, epochs = 10, verbose = 0, seed = 42)[1]['val_accuracy'])
  print(f'{branch} - {metric}')
  metrics_branch.append(metric)

In [ ]:
plt.plot(branch_list, metrics_branch, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('branch', fontsize=15)
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Лучшее количество branch = 4, хотя это скорее связано с размерами выходных изображений после MaxPool, которые будут 4x4 с 96 фильтрами.

### start_filters

In [ ]:
%%time

start_filters_list = [12, 18, 24, 36, 48]

metrics_start_filters = []

for start_filters in start_filters_list:
  metric = np.max(nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = start_filters, 
              start_kernel = 2, neurons = 2000, batch_size = 128, epochs = 10, verbose = 0, seed = 42)[1]['val_accuracy'])
  print(f'{start_filters} - {metric}')
  metrics_start_filters.append(metric)

In [ ]:
plt.plot(start_filters_list, metrics_start_filters, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('start_filters', fontsize=15)
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Лучшее количество start_filters = 24.

### start_kernel

In [ ]:
%%time

start_kernel_list = [1, 2, 3, 4, 5]

metrics_start_kernel = []

for start_kernel in start_kernel_list:
  metric = np.max(nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, 
              start_kernel = start_kernel, neurons = 2000, batch_size = 128, epochs = 10, verbose = 0, seed = 42)[1]['val_accuracy'])
  print(f'{start_kernel} - {metric}')
  metrics_start_kernel.append(metric)

In [ ]:
plt.plot(start_kernel_list, metrics_start_kernel, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('start_kernel', fontsize=15)
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Лучшее количество start_kernel = 2, хотя оно и чётное, оно даёт лучшую метрику.

### neurons

In [ ]:
%%time

neurons_list = [1000, 2000, 3000, 5000, 10000]

metrics_neurons = []

for neurons in neurons_list:
  metric = np.max(nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, 
              start_kernel = 2, neurons = neurons, batch_size = 128, epochs = 10, verbose = 0, seed = 42)[1]['val_accuracy'])
  print(f'{neurons} - {metric}')
  metrics_neurons.append(metric)

In [ ]:
plt.plot(neurons_list, metrics_neurons, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('neurons', fontsize=15)
plt.xscale('log')
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Лучшее количество neurons = 2000. При входных данных размером 4x4x24x2^4 = 6144.

### batch_size

In [ ]:
%%time

batch_size_list = [32, 64, 128, 256, 512]

metrics_batch_size = []

for batch_size in batch_size_list:
  metric = np.max(nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, 
              start_kernel = 2, neurons = 2000, batch_size = batch_size, epochs = 10, verbose = 0, seed = 42)[1]['val_accuracy'])
  print(f'{batch_size} - {metric}')
  metrics_batch_size.append(metric)

In [ ]:
plt.plot(batch_size_list, metrics_batch_size, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('batch_size', fontsize=15)
plt.xscale('log')
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Лучшее количество batch_size = 128.

### epochs

In [ ]:
metrics_epochs = nn_conv(x_train, x_test, y_train, y_test, branch = 4, start_filters = 24, 
              start_kernel = 2, neurons = 2000, batch_size = 128, epochs = 30, verbose = 1, seed = 42)[1]['val_accuracy']

Epoch 1/30
391/391 [==============================] - 40s 99ms/step - loss: 1.4638 - accuracy: 0.4688 - val_loss: 1.1414 - val_accuracy: 0.5962
Epoch 2/30
391/391 [==============================] - 38s 98ms/step - loss: 1.0138 - accuracy: 0.6412 - val_loss: 0.9531 - val_accuracy: 0.6627
Epoch 3/30
391/391 [==============================] - 38s 97ms/step - loss: 0.8267 - accuracy: 0.7085 - val_loss: 0.8549 - val_accuracy: 0.7088
Epoch 4/30
391/391 [==============================] - 38s 97ms/step - loss: 0.6991 - accuracy: 0.7556 - val_loss: 0.7912 - val_accuracy: 0.7269
Epoch 5/30
391/391 [==============================] - 38s 98ms/step - loss: 0.5831 - accuracy: 0.7951 - val_loss: 0.7492 - val_accuracy: 0.7431
Epoch 6/30
193/391 [=============>................] - ETA: 18s - loss: 0.4675 - accuracy: 0.8377

In [ ]:
plt.plot(list(range(1,11)), metrics_epochs, color = 'red')
plt.title('NN_accuracy', fontsize=20)
plt.xlabel('epochs', fontsize=15)
plt.ylabel('accuracy', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()

Но 9 эпохе нейронная сеть уже сходится. Дальнейшее обучение вызывает переобучение. Так как метрика на трейне стримится к 1.

#### Вывод:

Я постарался сделать необычную нелинейную свёрточную нейронную сеть, учитывая заканомерности изученной сети AlexNet, а точнее: сначала задаём большой размер ядра для свёртки, далее уменьшаем размер ядра свёртки, увиличиваем количество фильтров и используем связки, свёртка - пулинг. Далее выпремляем наш конечный тензор, прогоняем его через полносвязный нейронный слой и выводим на конечные 10 выходных нейронов наш ответ в вероятностном виде (от 0 до 1) с помощью функции активации softmax.

Идея моей нейронной сети, чтобы на каждом этапе свёртка - пулинг, нейронная сеть вспоминала изначальные данные той же размерности, но свёрнутые ядрами разных размеров. Такая компоновка данных на каждом этапе, должна давать более интересные резуьтаты, чем обычная линейная модель.

Я проэксперементировал с гиперпараметрами моей нейронной сети и сделал выводы для каждого гиперпараметра.